# Redes Neuronales Recurrentes aplicadas a la calidad del agua

De acuerdo al planteamiento pedido para la toma de datos en este caso la generacion de un dataset sintetico respecto a la calidad del agua se revisó el siguiente documento `https://www.bivica.org/files/normativa-calidad-agua.pdf` sobre la normativa nacional de los riquisitos del agua donde es posible determinar los campos fisicos, organolepticos, quimicos, parametros de desinfeccion, plaguicidas, radioactividad, microbiologicos y de control referencial, se tomo en cuenta los valores de control refencial y los requisitos fisicos dado que estos demostraran el proceso por el cual el agua pasa para asegurar su potabilidad teniendo como referencia la calidad del agua previo tratamiento y post tratamiento.

### Importacion de librerias

In [1]:
import numpy as np

### Generacion del dataset sintetico

In [2]:
def generate_water_quality_data(batch_size, n_steps):
    # Define parameters for water quality
    conductivity = 1500.0  # µS/cm
    alkalinity = 370.0  # mg/L CaCO3
    langelier_index = np.random.uniform(-0.5, 0.5, size=(batch_size, 1))  # Langelier index
    
    # Generate time steps
    time = np.linspace(0, 1, n_steps)
    
    # Generate water quality data
    conductivity_data = conductivity + 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    alkalinity_data = alkalinity + 0.05 * (np.random.rand(batch_size, n_steps) - 0.5)
    
    # Scale alkalinity based on Langelier index
    alkalinity_data *= (1 + langelier_index * 0.2)  # Adjust based on Langelier index
    
    # Additional parameters for water quality
    color_data = 15 + 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    turbidity_data = 5 + 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    dissolved_solids_data = 1000 + 10 * (np.random.rand(batch_size, n_steps) - 0.5)
    
    return np.concatenate([conductivity_data[..., np.newaxis], 
                            alkalinity_data[..., np.newaxis],
                            color_data[..., np.newaxis],
                            turbidity_data[..., np.newaxis],
                            dissolved_solids_data[..., np.newaxis]], axis=-1).astype(np.float32)

In [3]:
n_steps = 50
water_quality_data = generate_water_quality_data(43800, n_steps + 1)

# Split data into train, validation, and test sets
X_train, y_train = water_quality_data[:30660, :n_steps], water_quality_data[:30660, -1]
X_valid, y_valid = water_quality_data[30660:39480, :n_steps], water_quality_data[30660:39480, -1]
X_test, y_test = water_quality_data[39480:, :n_steps], water_quality_data[39480:, -1]
X_train.shape, y_train.shape

((30660, 50, 5), (30660, 5))

In [4]:
print (water_quality_data)

[[[1500.0383     379.4879      14.97905      5.036208   999.6854   ]
  [1500.0323     379.47836     15.036694     4.963944  1001.72864  ]
  [1500.0233     379.4977      15.015889     5.005746   997.35706  ]
  ...
  [1499.9835     379.49933     14.987739     5.009787  1001.355    ]
  [1500.0242     379.4991      14.971657     5.0204406  995.02673  ]
  [1499.9857     379.4749      14.955719     5.0488944 1003.96265  ]]

 [[1499.9543     351.49243     14.985538     4.9589853 1000.49927  ]
  [1500.0114     351.49252     14.980996     5.010379  1001.12506  ]
  [1500.0483     351.502       15.048388     5.0219073  996.7843   ]
  ...
  [1500.0289     351.50116     14.96382      5.0439444 1000.86176  ]
  [1500.0107     351.5188      14.995224     5.0192223  998.3963   ]
  [1500.011      351.5124      14.982451     4.969634   995.9462   ]]

 [[1500.0189     344.7967      14.984233     5.038854  1002.4518   ]
  [1500.0468     344.77658     14.989431     5.000339  1001.1082   ]
  [1500.0192     3